# Introduction

1. In the Previous project object, DS was able to easily associate domains, as we did not have association
requests.This would mean that the DS needs to submit assocation requests to domains to connect them with each other.

2. Project Invitation is a useful feature, but is very hard to implement, when in a distrubted setting, let us assume the DS is able to create projects on domains without any request/approval on the Project Itself.

3. The Data Scientist is able to create a project if and only if , all the domains could talk to each other. (i.e they have beeen previously associated)

4. This would mean that in our semi-Decentralized, leader based system , when ever a server would like to add an event/message to the project, it has to be sent to the leader, which is then broadcasted to all the other servers.

5. For other situations, for example when a server would like to asset metadata of another server in a Multi Domain User Code request, they could directly contact the server, to retrieve the info, instead of going through the leader.

6. This would require us to create a Full Mesh Network Topology, where each server is connected to each other.

In [ ]:
# syft absolute
import syft as sy
from syft.abstract_server import ServerType
from syft.service.network.server_peer import ServerPeer

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082

# Launch servers

We will begin by launching two domain servers and an enclave server.

In [ ]:
canada_server = sy.orchestra.launch(
    name="canada-domain",
    port=CANADA_DOMAIN_PORT,
    dev_mode=True,
)
italy_server = sy.orchestra.launch(
    name="italy-domain",
    port=ITALY_DOMAIN_PORT,
    dev_mode=True,
)

In [ ]:
ds_canada_client = canada_server.login(
    email="sheldon@caltech.edu", password="changethis"
)
ds_italy_client = italy_server.login(email="sheldon@caltech.edu", password="changethis")

assert ds_canada_client.metadata.server_type == ServerType.DOMAIN
assert ds_italy_client.metadata.server_type == ServerType.DOMAIN

# Create Assocation Requests from DS

In [ ]:
canada_server_peer = ServerPeer.from_client(ds_canada_client)
canada_server_peer

In [ ]:
italy_server_peer = ServerPeer.from_client(ds_italy_client)
italy_server_peer

In [ ]:
canada_conn_req = ds_canada_client.api.services.network.add_peer(italy_server_peer)
canada_conn_req

In [ ]:
italy_conn_req = ds_italy_client.api.services.network.add_peer(canada_server_peer)
italy_conn_req

# Data Owners Login and Approve the Association Requests

In [ ]:
do_canada_client = canada_server.login(
    email="info@openmined.org", password="changethis"
)
do_italy_client = italy_server.login(email="info@openmined.org", password="changethis")

In [ ]:
do_canada_client.requests

In [ ]:
do_canada_client.requests[0].approve()

In [ ]:
assert do_canada_client.peers[0].id == do_italy_client.id
do_canada_client.peers

In [ ]:
do_italy_client.requests

In [ ]:
assert len(do_italy_client.api.services.request.get_all()) == 1
do_italy_client.requests[0].approve()

In [ ]:
assert do_italy_client.peers[0].id == do_canada_client.id
do_italy_client.peers

In [ ]:
# syft absolute
from syft.service.network.utils import check_route_reachability

In [ ]:
check_route_reachability([ds_canada_client, ds_italy_client])

# Cleanup local domain servers

In [ ]:
if canada_server.deployment_type.value == "python":
    canada_server.land()

if italy_server.deployment_type.value == "python":
    italy_server.land()